In [1]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="2" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *

# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *
from keras_self_attention import SeqSelfAttention

Using TensorFlow backend.
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## History

In [34]:
log_dir = './logs/MLP_2dense_0718/'
history = pickle.load(open(log_dir+'MLP_3dense_gruatt_H.pkl','rb'))
keys = list(history.keys())
keys

['val_loss', 'val_bin_acc', 'loss', 'bin_acc']

In [35]:
history_df = pd.DataFrame(data={keys[0]:history[keys[0]], keys[1]:history[keys[1]],
                               keys[2]:history[keys[2]], keys[3]:history[keys[3]]})
history_df.to_excel('data/tree-rep-profiles_o2o/encEXP_MLP_history_0718.xlsx',index=True)
history_df

,val_loss,val_bin_acc,loss,bin_acc
0,1.949592,0.545345,0.679174,0.717514
1,0.628379,0.615690,0.540690,0.775704
2,0.767312,0.611486,0.476352,0.802738
3,0.774234,0.594588,0.477878,0.801393
4,1.354254,0.556720,0.436396,0.846643
5,0.930971,0.562779,0.553420,0.780035
6,0.620704,0.601777,0.491871,0.789665
7,0.601158,0.628626,0.576487,0.716091
8,0.687972,0.592990,0.578266,0.716544
9,0.668036,0.604354,0.504097,0.766543


## TF Events

In [12]:
event_file = log_dir+'events.out.tfevents.1562415044.superpc4'
for event in tf.train.summary_iterator(event_file):
    for value in event.summary.value:
        print(value.tag)
        break
#     break

val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
val_loss
val_bin_acc
loss
bin_acc
lr
v

In [14]:
# e，即event，代表某一個batch的日誌記錄
for e in tf.train.summary_iterator(event_file):
    # v，即value，代表這個batch的某個已記錄的觀測值，loss或者accuracy
    for v in e.summary.value:
        if v.tag == 'loss' or v.tag == 'bin_acc':
            print(v.tag,v.simple_value)

loss 0.946535050868988
bin_acc 0.7701650857925415
loss 0.3709505796432495
bin_acc 0.8308416604995728
loss 0.34396806359291077
bin_acc 0.8539522886276245
loss 0.3589724004268646
bin_acc 0.8503860235214233
loss 0.3080110251903534
bin_acc 0.8667504787445068
loss 0.32501688599586487
bin_acc 0.8593385815620422
loss 0.31698301434516907
bin_acc 0.8747698068618774
loss 0.3775538504123688
bin_acc 0.8374796509742737
loss 0.3339521586894989
bin_acc 0.8584908843040466
loss 0.32204964756965637
bin_acc 0.8647983074188232
loss 0.35764580965042114
bin_acc 0.8471481800079346
loss 0.31919118762016296
bin_acc 0.8746025562286377
loss 0.3112717270851135
bin_acc 0.8738447427749634
loss 0.31367820501327515
bin_acc 0.865882933139801
loss 0.33277350664138794
bin_acc 0.858725368976593
loss 0.3152080178260803
bin_acc 0.8607260584831238
loss 0.3059806823730469
bin_acc 0.8709530830383301
loss 0.33169272541999817
bin_acc 0.8512396812438965
loss 0.34347817301750183
bin_acc 0.8580807447433472
loss 0.3191557228565216
